# Coursera - Applied Data Science with Capstone

## Week 3 Assignment - Part 1

### Setup - import Packages

In [51]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests
import lxml

### Use bs4 to extract table of Toronto postcodes from Wikipedia

In [4]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [56]:
soup = BeautifulSoup(url, "html.parser")

In [57]:
my_table = soup.find('table',{'class':'wikitable sortable'})

### Convert Table to dataframe, clean and remove 'Not assigned' entries

In [64]:
df = pd.read_html(str(my_table))[0]

In [79]:
df.columns = df.iloc[0]

In [82]:
df = df.drop(0,axis=0)

In [91]:
df_clean = df[(df['Borough'] != 'Not assigned') & (df['Neighbourhood'] != 'Not assigned')]

In [98]:
df_clean.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


### Group by Postcode and append Neighbourhoods into one row

In [101]:
toronto = df_clean.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))

In [105]:
toronto = pd.DataFrame(toronto)

In [109]:
toronto = toronto.reset_index()

In [113]:
toronto.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
